In [ ]:
capacity = (8,5,3) 
# Maximum capacities of 3 jugs -> x,y,z
x = capacity[0]
y = capacity[1]
z = capacity[2]

memory = {}
ans = []

def get_all_states(state):
	a = state[0]
	b = state[1]
	c = state[2]

	if(a==4 and b==4):
		ans.append(state)
		return True

	if((a,b,c) in memory):
		return False

	memory[(a,b,c)] = 1

	if(a>0):
		if(a+b<=y):
			if( get_all_states((0,a+b,c)) ):
				ans.append(state)
				return True
		else:
			if( get_all_states((a-(y-b), y, c)) ):
				ans.append(state)
				return True
		if(a+c<=z):
			if( get_all_states((0,b,a+c)) ):
				ans.append(state)
				return True
		else:
			if( get_all_states((a-(z-c), b, z)) ):
				ans.append(state)
				return True

	if(b>0):
		if(a+b<=x):
			if( get_all_states((a+b, 0, c)) ):
				ans.append(state)
				return True
		else:
			if( get_all_states((x, b-(x-a), c)) ):
				ans.append(state)
				return True
		if(b+c<=z):
			if( get_all_states((a, 0, b+c)) ):
				ans.append(state)
				return True
		else:
			if( get_all_states((a, b-(z-c), z)) ):
				ans.append(state)
				return True

	if(c>0):
		if(a+c<=x):
			if( get_all_states((a+c, b, 0)) ):
				ans.append(state)
				return True
		else:
			if( get_all_states((x, b, c-(x-a))) ):
				ans.append(state)
				return True
		if(b+c<=y):
			if( get_all_states((a, b+c, 0)) ):
				ans.append(state)
				return True
		else:
			if( get_all_states((a, y, c-(y-b))) ):
				ans.append(state)
				return True

	return False

initial_state = (8,0,0)
print("3 JUG PROBLEM: \n\n(A, B, C)\n")
get_all_states(initial_state)
ans.reverse()
for i in ans:
	print(i)

3 JUG PROBLEM: 

(A, B, C)

(8, 0, 0)
(3, 5, 0)
(0, 5, 3)
(5, 0, 3)
(5, 3, 0)
(2, 3, 3)
(2, 5, 1)
(7, 0, 1)
(7, 1, 0)
(4, 1, 3)
(4, 4, 0)


In [ ]:
from copy import deepcopy
from collections import deque
import sys
import time

print("MISSIONARIES and CANNIBALS PROBLEM:\n")

class State(object):
  def __init__(self, missionaries, cannibals, boats):
    self.missionaries = missionaries
    self.cannibals = cannibals
    self.boats = boats
  
  def successors(self):
    if self.boats == 1:
      sgn = -1
      direction = "from the original shore to the new shore"
    else:
      sgn = 1
      direction = "back from the new shore to the original shore"
    for m in range(3):
      for c in range(3):
        newState = State(self.missionaries+sgn*m, self.cannibals+sgn*c, self.boats+sgn*1);
        if m+c >= 1 and m+c <= 2 and newState.isValid():   
          action = "Take %d missionaries and %d cannibals %s. %r" % ( m, c, direction, newState) 
          yield action, newState
            
  def isValid(self):
    if self.missionaries < 0 or self.cannibals < 0 or self.missionaries > 3 or self.cannibals > 3 or (self.boats != 0 and self.boats != 1):
      return False   
    if self.cannibals > self.missionaries and self.missionaries > 0:    
      return False
    if self.cannibals < self.missionaries and self.missionaries < 3:    
      return False
    return True

  def is_goal_state(self):
    return self.cannibals == 0 and self.missionaries == 0 and self.boats == 0

  def __repr__(self):
    return "< State (%d, %d, %d) >" % (self.missionaries, self.cannibals, self.boats)



class Node(object):  
  def __init__(self, parent_node, state, action, depth):
    self.parent_node = parent_node
    self.state = state
    self.action = action
    self.depth = depth
  
  def expand(self):
    for (action, succ_state) in self.state.successors():
      succ_node = Node(
                       parent_node=self,
                       state=succ_state,
                       action=action,
                       depth=self.depth + 1)
      yield succ_node
  
  def extract_solution(self):
    solution = []
    node = self
    while node.parent_node is not None:
      solution.append(node.action)
      node = node.parent_node
    solution.reverse()
    return solution


def breadth_first_tree_search(initial_state):
  initial_node = Node(
                      parent_node=None,
                      state=initial_state,
                      action=None,
                      depth=0)
  fifo = deque([initial_node])
  num_expansions = 0
  max_depth = -1
  while True:
    if not fifo:
      return None
    node = fifo.popleft()
    if node.depth > max_depth:
      max_depth = node.depth
    if node.state.is_goal_state():
      solution = node.extract_solution()
      return solution
    num_expansions += 1
    fifo.extend(node.expand())


def usage():
  print >> sys.stderr, "usage:"
  print >> sys.stderr, "    %s" % sys.argv[0]
  raise SystemExit(2)


def main():
  initial_state = State(3,3,1)
  solution = breadth_first_tree_search(initial_state)
  if solution is None:
    print("no solution")
  else:
    print("Solution (%d steps):" % len(solution))
    for step in solution:
      print("%s" % step)


if __name__ == "__main__":
  main()

MISSIONARIES and CANNIBALS PROBLEM:

Solution (11 steps):
Take 0 missionaries and 2 cannibals from the original shore to the new shore. < State (3, 1, 0) >
Take 0 missionaries and 1 cannibals back from the new shore to the original shore. < State (3, 2, 1) >
Take 0 missionaries and 2 cannibals from the original shore to the new shore. < State (3, 0, 0) >
Take 0 missionaries and 1 cannibals back from the new shore to the original shore. < State (3, 1, 1) >
Take 2 missionaries and 0 cannibals from the original shore to the new shore. < State (1, 1, 0) >
Take 1 missionaries and 1 cannibals back from the new shore to the original shore. < State (2, 2, 1) >
Take 2 missionaries and 0 cannibals from the original shore to the new shore. < State (0, 2, 0) >
Take 0 missionaries and 1 cannibals back from the new shore to the original shore. < State (0, 3, 1) >
Take 0 missionaries and 2 cannibals from the original shore to the new shore. < State (0, 1, 0) >
Take 0 missionaries and 1 cannibals back